In [ ]:
import pandas as pd
import pickle
import os
import re

from functools import reduce

In [ ]:
import sys
sys.path.insert(0, '/media/andrii/earth/Katia/CS_MasterThesis/Red_links_Project_for_Wiki_draft/py')
import utils

In [ ]:
PATH_TO_DATA = '/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/'

In [ ]:
# get all the candidates with true lable
df_with_all_candidates = pd.read_csv(PATH_TO_DATA+'ukredlinks_with_all_candidates&true_label', sep='^')

In [ ]:
df_with_all_candidates.head()

# 1. BabelNet results

In [ ]:
BN_results = pd.read_csv(PATH_TO_DATA+'BN_ukredlinks_wiki_evaluated.csv', encoding = 'UTF-8')

In [ ]:
BN_results.head()

In [ ]:
df_with_BN = pd.merge(df_with_all_candidates, BN_results, how='left', left_on='red_link', right_on='red link name')
df_with_BN.shape

In [ ]:
df_with_BN.columns
df_with_BN = df_with_BN[['red_link','candidate', 'true_label', 'EN BN']]

# strip all string values in a dataframe
df_with_BN_obj = df_with_BN.select_dtypes(['object'])
df_with_BN[df_with_BN_obj.columns] = df_with_BN_obj.apply(lambda x: x.str.strip())

bn_results = []
for index, row in df_with_BN.iterrows():
    bn_result = row['candidate'] == row['EN BN']
    bn_results.append(bn_result)
BN_df = pd.DataFrame(bn_results)

with_BN_df = pd.concat([df_with_BN, BN_df], axis=1)
with_BN_df.columns = ['red_link_name', 'candidate', 'true_label', 'EN BN', 'BN_results']
with_BN_df = with_BN_df[['red_link_name', 'candidate', 'true_label', 'BN_results']]

with_BN_df['BN_results'] = with_BN_df['BN_results'].replace({False: '0'})
with_BN_df['BN_results'] = with_BN_df['BN_results'].replace({True: '1'})

with_BN_df = with_BN_df.reset_index(level=0, drop=True)
print(with_BN_df.shape)
with_BN_df.head()

# 2. Jaccard similarity on incoming links

In [ ]:
# get all the candidates with true lable
df_with_all_candidates = pd.read_csv(PATH_TO_DATA+'ukredlinks_with_all_candidates&true_label.csv', sep='^')
df_with_all_candidates.head()

In [ ]:
candidates = pd.read_csv(PATH_TO_DATA+'uk_red_links_results_all_candidates.csv', sep='^')
candidates.head()

In [ ]:
all_results_scores = {}
all_results_list = [[]] * candidates.shape[0]

for i in range(candidates.shape[0]):
    if candidates['en_similar'][i] == '[]':
        all_results_scores[candidates['red_link_name'][i]] = []
        all_results_list[i] = []
    else:
        res = utils.extract_score(candidates['en_similar'][i])
        all_results_scores[candidates['red_link_name'][i]] = res
        all_results_list[i] = res

df = pd.DataFrame.from_dict(all_results_scores, orient = 'index')

rl_with_score = df.stack().reset_index(level=0, drop=True).to_frame(name='jaccard_score')
rl_with_score = rl_with_score.reset_index(level=0, drop=True)
df = pd.concat([with_BN_df, rl_with_score], axis=1)
df.head()

# 3. Jaccard scores on concurrent links 

## 3.1. get scores on concurrent links for uk red links

## 3.2. get scores on concurrent links for en candidates

## 3.3. calculate scores and save to file

In [ ]:
candidate_pairs = pd.read_csv(PATH_TO_DATA_UK+'red_links_uk_with_features_with_uniqueid.csv', sep='^')

In [ ]:
concur_folders = [f for f in os.listdir(PATH_TO_DATA_UK) if re.match(r'concurrent_scores', f)]

In [ ]:
concurrent_scores_1_df = utils.files_to_df(PATH_TO_DATA_UK+'concurrent_scores/')
concurrent_scores_2_df = utils.files_to_df(PATH_TO_DATA_UK+'concurrent_scores_2/')
concurrent_scores_3_df = utils.files_to_df(PATH_TO_DATA_UK+'concurrent_scores_3/')
concurrent_scores_4_df = utils.files_to_df(PATH_TO_DATA_UK+'concurrent_scores_4/')

dfs = [concurrent_scores_1_df, concurrent_scores_2_df, concurrent_scores_3_df, concurrent_scores_4_df]

concurrent_scores = pd.concat(dfs)
concurrent_scores = concurrent_scores.drop_duplicates().reset_index(drop=True)
concurrent_scores = concurrent_scores.sort_values(by=['pair_id'])
print(concurrent_scores.shape)
concurrent_scores.head()

In [ ]:
features_with_concur_scores =\
    pd.merge(candidate_pairs, concurrent_scores, how='right', left_on=['pair_unique_id'], right_on=['pair_id'])

features_with_concur_scores =\
    features_with_concur_scores[['pair_unique_id', 'red_link_name', 'candidate', 'true_label', 'BN_results',\
                                'jaccard_score', 'levenstein_score', 'concur_score']]

features_with_concur_scores.head()

In [ ]:
features_with_concur_scores.to_csv(PATH_TO_DATA_UK+'red_links_uk_with_3features.csv', sep="^", header=True, index=False)

# 4. Levenshtein edit distance scores on titles

In [ ]:
df['levenstein_score'] = df.apply(utils.calculate_norm_levenstein_df, axis=1)
df.to_csv(PATH_TO_DATA+'red_links_uk_with_3features.csv', header=True, index=False)
df.head()